In [32]:
salario_mensal_bruto=[1_903.98, 2_826.66, 3_751.06, 4_664.68, 8_333.34, 16_666.67, 25_000, 30_000]


In [35]:
def mostrar_descontos_mensais(funcao):
    return [(f'R$ {funcao(salario):,.2f}', f'{funcao(salario) / salario:.2%}') for salario in salario_mensal_bruto]

In [34]:
def imposto_de_renda_mensal(salario_mensal:float) -> float:
    """
    Calcula imposto de renda PF de acordo com faixa do ano de 2019
    https://impostoderenda2019.net.br/tabela-imposto-de-renda-2019/
    
    """
    saldo = salario_mensal
    aliquotas_faixas = {0.275: 4_664.68, 0.225: 3_751.06, 0.15: 2_826.66, 0.075: 1_903.98, 0: 0}
    ir = 0
    desconto_simplificado = 0.20  # 20% de desconto simplificado
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        ir += delta * aliquota * (1 - desconto_simplificado)
        saldo = min(saldo, faixa)
    return ir


In [36]:
mostrar_descontos_mensais(imposto_de_renda_mensal)


[('R$ 0.00', '0.00%'),
 ('R$ 55.36', '1.96%'),
 ('R$ 166.29', '4.43%'),
 ('R$ 330.74', '7.09%'),
 ('R$ 1,137.85', '13.65%'),
 ('R$ 2,971.18', '17.83%'),
 ('R$ 4,804.51', '19.22%'),
 ('R$ 5,904.51', '19.68%')]

In [37]:
def fgts_empregador(salario_mensal: float)-> float:
    """
    Calcula fgts a ser pago de imposto pelo empregador em 1 ano. 
    Aliquota de 8% + 40% disso em caso de demissão com justa causa -> 11,2%
    
    
    :param salario_mensal: salário mensal bruto do empregado
    :return: float
    """
    return salario_mensal * 0.112


In [38]:
mostrar_descontos_mensais(fgts_empregador)

[('R$ 213.25', '11.20%'),
 ('R$ 316.59', '11.20%'),
 ('R$ 420.12', '11.20%'),
 ('R$ 522.44', '11.20%'),
 ('R$ 933.33', '11.20%'),
 ('R$ 1,866.67', '11.20%'),
 ('R$ 2,800.00', '11.20%'),
 ('R$ 3,360.00', '11.20%')]

In [39]:
def inss_empregado(salario_mensal):
    """
    Calcula inss para 2019: 1.751,81
    :param salario_mensal: 
    :return: 
    """
    teto = 5_839.45
    if salario_mensal < 1_751.81:
        return salario_mensal * 0.08
    elif salario_mensal < 2_919.72:
        return salario_mensal * 0.09
   
    return min(salario_mensal, teto) * 0.11


In [40]:

mostrar_descontos_mensais(inss_empregado)

[('R$ 171.36', '9.00%'),
 ('R$ 254.40', '9.00%'),
 ('R$ 412.62', '11.00%'),
 ('R$ 513.11', '11.00%'),
 ('R$ 642.34', '7.71%'),
 ('R$ 642.34', '3.85%'),
 ('R$ 642.34', '2.57%'),
 ('R$ 642.34', '2.14%')]

In [42]:
def imposto_de_renda_anual(salario):
    """
    Calcula imposto de renda anual considerando Décimo terceiro e terço de ferias
    :param salario: 
    :return: 
    """
    ir = imposto_de_renda_mensal(salario) * 11
    salario_de_ferias = salario + salario / 3
    ir += imposto_de_renda_mensal(salario_de_ferias - inss_empregado(salario_de_ferias))
    ir += imposto_de_renda_mensal(salario - inss_empregado(salario)) / 0.80  # Tributação exlusiva de décimo terceiro
    return ir


In [45]:
def mostrar_descontos_anuais(funcao):
    salarios = salario_mensal_bruto
    salarios_anuais = [s * 13 + s / 3 for s in salario_mensal_bruto]
    for tpl in [
        (f'Salário Mensal R$ {salario:,.2f}', f' Salário Anual R$ {salario_anual:,.2f}', f'R$ {funcao(salario):,.2f}', f'{funcao(salario) / salario_anual:.2%}') for
        salario,  salario_anual in zip(salarios, salarios_anuais)]:
        print(*tpl, sep=' - ')


In [46]:
mostrar_descontos_anuais(imposto_de_renda_anual)

Salário Mensal R$ 1,903.98 -  Salário Anual R$ 25,386.40 - R$ 24.37 - 0.10%
Salário Mensal R$ 2,826.66 -  Salário Anual R$ 37,688.80 - R$ 777.77 - 2.06%
Salário Mensal R$ 3,751.06 -  Salário Anual R$ 50,014.13 - R$ 2,267.47 - 4.53%
Salário Mensal R$ 4,664.68 -  Salário Anual R$ 62,195.73 - R$ 4,467.62 - 7.18%
Salário Mensal R$ 8,333.34 -  Salário Anual R$ 111,111.20 - R$ 15,369.61 - 13.83%
Salário Mensal R$ 16,666.67 -  Salário Anual R$ 222,222.27 - R$ 40,272.38 - 18.12%
Salário Mensal R$ 25,000.00 -  Salário Anual R$ 333,333.33 - R$ 65,175.14 - 19.55%
Salário Mensal R$ 30,000.00 -  Salário Anual R$ 400,000.00 - R$ 80,116.81 - 20.03%


In [47]:
def inss_anual(salario):
    """
    Calcula valor total decontado do empregado anualmente
    :param salario: 
    :return: 
    """
    ONZE_MESES_MAIS_DECIMO_TERCEIRO = 12
    inss = inss_empregado(salario) * ONZE_MESES_MAIS_DECIMO_TERCEIRO
    salario_ferias = salario + salario / 3
    return inss + inss_empregado(salario_ferias)


In [49]:
mostrar_descontos_anuais(inss_anual)

Salário Mensal R$ 1,903.98 -  Salário Anual R$ 25,386.40 - R$ 2,284.78 - 9.00%
Salário Mensal R$ 2,826.66 -  Salário Anual R$ 37,688.80 - R$ 3,467.37 - 9.20%
Salário Mensal R$ 3,751.06 -  Salário Anual R$ 50,014.13 - R$ 5,501.55 - 11.00%
Salário Mensal R$ 4,664.68 -  Salário Anual R$ 62,195.73 - R$ 6,799.72 - 10.93%
Salário Mensal R$ 8,333.34 -  Salário Anual R$ 111,111.20 - R$ 8,350.41 - 7.52%
Salário Mensal R$ 16,666.67 -  Salário Anual R$ 222,222.27 - R$ 8,350.41 - 3.76%
Salário Mensal R$ 25,000.00 -  Salário Anual R$ 333,333.33 - R$ 8,350.41 - 2.51%
Salário Mensal R$ 30,000.00 -  Salário Anual R$ 400,000.00 - R$ 8,350.41 - 2.09%


In [27]:
def impostos_anuais(salario):
    return imposto_de_renda_anual(salario)+inss_empregado(salario)

In [54]:
def resumo_empregado(salario):
    """
    Retorna tupla com (Salario Mensal, Salario Mensal liquido,Salario Anual Liquido, Salario Liquido +fgts,Salario Bruto Anual , Impostos, FGTS, 
    :param funcao: 
    :return: 
    """

    salario_anual_bruto = salario * 13 + salario / 3
    impostos = impostos_anuais(salario)
    fgts_anual = fgts_empregador(salario)
    fgts_anual = fgts_anual * 13 + fgts_anual / 3
    salario_mensal_liquido = salario - imposto_de_renda_mensal(salario) - fgts_empregador(salario) - inss_empregado(salario)
    salario_anual_menos_impostos = salario_anual_bruto - impostos
    return salario, salario_mensal_liquido, salario_anual_menos_impostos, salario_anual_menos_impostos + fgts_anual, salario_anual_bruto, impostos, fgts_anual,



In [57]:
for salario in salario_mensal_bruto:
    print(*map(lambda v: f'{v:<15,.2f}', resumo_empregado(salario)), sep=' - ')

1,903.98        - 1,519.38        - 25,190.67       - 28,033.95       - 25,386.40       - 195.73          - 2,843.28       
2,826.66        - 2,200.31        - 36,656.63       - 40,877.78       - 37,688.80       - 1,032.17        - 4,221.15       
3,751.06        - 2,752.04        - 47,334.05       - 52,935.63       - 50,014.13       - 2,680.09        - 5,601.58       
4,664.68        - 3,298.38        - 57,215.00       - 64,180.92       - 62,195.73       - 4,980.74        - 6,965.92       
8,333.34        - 5,619.82        - 95,099.25       - 107,543.71      - 111,111.20      - 16,011.95       - 12,444.45      
16,666.67       - 11,186.49       - 181,307.55      - 206,196.45      - 222,222.27      - 40,914.72       - 24,888.89      
25,000.00       - 16,753.15       - 267,515.85      - 304,849.18      - 333,333.33      - 65,817.48       - 37,333.33      
30,000.00       - 20,093.15       - 319,240.85      - 364,040.85      - 400,000.00      - 80,759.15       - 44,800.00      


In [58]:
faturamento_bruto = [13 * s + s / 3 + fgts_empregador(s) for s in salario_mensal_bruto]



In [62]:
[f'{v:,.2f}' for v in faturamento_bruto]

['25,599.65',
 '38,005.39',
 '50,434.25',
 '62,718.18',
 '112,044.53',
 '224,088.93',
 '336,133.33',
 '403,360.00']

In [63]:
def custos_simples_anexo_iii(faturamento_anual, contador=89, prolabore=1_100) -> float:
    """
    Calcula custos de impostos para empresa simples de anexo III
    DAS, Prólabore e Contador
    
    Aliquotas 2019: https://www.contabilizei.com.br/contabilidade-online/anexo-3-simples-nacional/
    :param faturamento_anual: 
    :param contador: 
    :param prolabore: 
    :return: 
    """
    saldo = faturamento_anual
    aliquotas_faixas = {0.33: 3_600_000, 0.21: 1_800_000, 0.16: 720_000, 0.135: 360_000, 0.112: 180_000, 0.06: 0}
    imposto = 0
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        imposto += delta * aliquota
        saldo = min(saldo, faixa)
    inss = prolabore * 0.11
    return imposto + 12 * (contador + inss)


In [64]:
def resumo_pj(faturamento):
    custo_anual = custos_simples_anexo_iii(faturamento)
    return faturamento / 12, (faturamento - custo_anual) / 12, faturamento - custo_anual, faturamento 



In [65]:
for faturamento in faturamento_bruto:
    print(*map(lambda v: f'{v:<15,.2f}', resumo_pj(faturamento)), sep=' - ')

2,133.30        - 1,795.31        - 21,543.67       - 25,599.65      
3,167.12        - 2,767.09        - 33,205.06       - 38,005.39      
4,202.85        - 3,740.68        - 44,888.20       - 50,434.25      
5,226.51        - 4,702.92        - 56,435.09       - 62,718.18      
9,337.04        - 8,566.82        - 102,801.86      - 112,044.53     
18,674.08       - 17,152.58       - 205,830.97      - 224,088.93     
28,011.11       - 25,443.87       - 305,326.40      - 336,133.33     
33,613.33       - 30,335.53       - 364,026.40      - 403,360.00     


In [66]:
for salario, faturamento in zip(salario_mensal_bruto, faturamento_bruto):
    print(f'Salário Mensal {salario:,.2f}')
    clt = resumo_empregado(salario)[2]
    print(f'CLT: R$ {clt :<15,.2f}', sep='')
    pj = resumo_pj(faturamento)[2]
    print(f'PJ: R$ {pj :<15,.2f}')
    print(f'DIFF: R${pj - clt :<15,.2f}')
    print()


Salário Mensal 1,903.98
CLT: R$ 25,190.67      
PJ: R$ 21,543.67      
DIFF: R$-3,647.00      

Salário Mensal 2,826.66
CLT: R$ 36,656.63      
PJ: R$ 33,205.06      
DIFF: R$-3,451.57      

Salário Mensal 3,751.06
CLT: R$ 47,334.05      
PJ: R$ 44,888.20      
DIFF: R$-2,445.85      

Salário Mensal 4,664.68
CLT: R$ 57,215.00      
PJ: R$ 56,435.09      
DIFF: R$-779.91        

Salário Mensal 8,333.34
CLT: R$ 95,099.25      
PJ: R$ 102,801.86     
DIFF: R$7,702.61       

Salário Mensal 16,666.67
CLT: R$ 181,307.55     
PJ: R$ 205,830.97     
DIFF: R$24,523.42      

Salário Mensal 25,000.00
CLT: R$ 267,515.85     
PJ: R$ 305,326.40     
DIFF: R$37,810.55      

Salário Mensal 30,000.00
CLT: R$ 319,240.85     
PJ: R$ 364,026.40     
DIFF: R$44,785.55      

